In [0]:
dbutils.fs.mkdirs("/Volumes/databricks-pokemon/pokemon/pokemon_raw/pokemons_details/")

In [0]:
%python
# Install the tqdm module
%pip install tqdm

import datetime  # Importa o módulo datetime para manipulação de datas e horários
import requests  # Importa a biblioteca requests para fazer requisições HTTP
import json  # Importa o módulo json para manipulação de dados JSON
from tqdm import tqdm  # Importa a biblioteca tqdm para exibir barras de progresso
from multiprocessing import Pool  # Importa Pool da biblioteca multiprocessing para processamento paralelo

# Função que faz requisição a um URL, obtém os dados e salva em um arquivo JSON
def get_and_save(url):
    data = requests.get(url).json()  # Faz a requisição GET para o URL e obtém os dados em formato JSON
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Obtém a data e hora atuais formatadas
    
    # Define o caminho do arquivo onde os dados serão salvos (possível erro devido ao caractere ':' em nomes de arquivos)
    filename = f"/Volumes/databricks-pokemon/pokemon/pokemon_raw/pokemons_details/{data['id']}.{now}.json"
    
    # Abre o arquivo no modo escrita e salva os dados JSON
    with open(filename, "w") as open_file:
        json.dump(data, open_file)

# Obtém a tabela do Databricks como um DataFrame Spark
df = spark.table("`databricks-pokemon`.pokemon.pokemon_bronze")

# Seleciona a coluna "url", converte para um DataFrame Pandas e extrai os valores como uma lista
urls = df.select('url').toPandas()['url'].tolist()

# Cria um pool de 4 processos para execução paralela das requisições
with Pool(4) as p:
    print(p.map(get_and_save, urls))  # Mapeia a função "get_and_save" para cada URL na lista usando multiprocessing

In [0]:
df = spark.read.json("/Volumes/databricks-pokemon/pokemon/pokemon_raw/pokemons_details/")

display(df)
                     